In [1]:
import tensorflow as tf
import numpy as np
import csv
from PIL import Image
import glob
from sklearn.cross_validation import train_test_split

/usr/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#import tr_label
tr_label_file = './data_island_in_the_sun/labels_training.csv'
tr_label = open(tr_label_file, 'rt')
tr_read = csv.reader(tr_label, delimiter=',', quoting=csv.QUOTE_NONE)
tr_label_list = list(tr_read)
y_data = np.array(tr_label_list)[1:].astype('int')
#print(y_data)
y_list = y_data[:,1:]

num_data = y_list.shape[0]

y_ = np.zeros([num_data,2])

for i in range(num_data):
    y_[i,y_list[i][0]] = 1

In [11]:
#import images
filelist = ["" for x in range(1500)]

for i in range(0, 1500):
    path = "./data_island_in_the_sun/train_data/" + str(i) + ".tif"
    filelist[i] = path

x_data = np.array([np.array(Image.open(fname)) for fname in filelist])


In [12]:
#prepocess 
x_list = x_data
x_list.shape

(1500, 101, 101, 3)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x_list, y_, test_size=0.2, random_state=0)
# X_train.shape, y_train.shape
# X_test.shape, y_test.shape

num_train = X_train.shape[0]
num_valid = X_test.shape[0]


In [28]:
# Training Parameters
learning_rate = 0.00001
num_epochs = 900
batch_size = 100
display_step = 100
img_width = 101


# Network Parameters
num_input = img_width*img_width*3 # data input (img shape: 101 * 101 * 3)
num_classes = 2 # total classes (0/1)
dropout = 0.75 # Dropout, probability to keep units



In [29]:
# tf Graph input
X = tf.placeholder(tf.float32, [None, img_width,img_width,3])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [30]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    
    # x = tf.reshape(x, shape=[-1, img_width, img_width, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.contrib.layers.flatten(conv2)
    # fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [31]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([676*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    num_st = num_train/batch_size
    
    for i in range(num_epochs):
        for step in range(num_st):
            si,ei = step*batch_size,(step+1)*batch_size 
            batch_x, batch_y = X_train[si:ei ,:], y_train[si:ei,:]
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
            if (i*num_st+step) % display_step == 0:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y,
                                                                     keep_prob: 1.0})
                print("Step " + str(i*num_st+step) + ", Minibatch Loss= " + \
                      "{:.4f}".format(loss) + ", Training Accuracy= " + \
                      "{:.3f}".format(acc))

                print("Testing Accuracy:", \
                    sess.run(accuracy, feed_dict={X: X_test,
                                                  Y: y_test,
                                                  keep_prob: 1.0}))                

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test,
                                      keep_prob: 1.0}))

Step 0, Minibatch Loss= 51546320.0000, Training Accuracy= 0.350
('Testing Accuracy:', 0.36666664)
Step 100, Minibatch Loss= 15040747.0000, Training Accuracy= 0.650
('Testing Accuracy:', 0.58999997)
Step 200, Minibatch Loss= 18376028.0000, Training Accuracy= 0.560
('Testing Accuracy:', 0.60666668)
Step 300, Minibatch Loss= 16259370.0000, Training Accuracy= 0.650
('Testing Accuracy:', 0.60666668)
Step 400, Minibatch Loss= 12838532.0000, Training Accuracy= 0.700
('Testing Accuracy:', 0.60666668)
Step 500, Minibatch Loss= 13901176.0000, Training Accuracy= 0.580
('Testing Accuracy:', 0.61000001)
Step 600, Minibatch Loss= 13909667.0000, Training Accuracy= 0.680
('Testing Accuracy:', 0.62)
Step 700, Minibatch Loss= 10060928.0000, Training Accuracy= 0.700
('Testing Accuracy:', 0.61666667)
Step 800, Minibatch Loss= 13434760.0000, Training Accuracy= 0.600
('Testing Accuracy:', 0.62333333)
Step 900, Minibatch Loss= 11719424.0000, Training Accuracy= 0.710
('Testing Accuracy:', 0.62333333)
Step 100

In [111]:
batch_x.shape

(50, 101, 101, 3)

In [58]:
batch_y.shape

(50, 2)